# 简介

[官方文档](https://dormousehole.readthedocs.io/en/latest/signals.html)

Flask框架中的信号基于blinker，其主要就是让开发者可是在flask请求过程中定制一些用户行为，如记录用户行为，登录记录等，和请求扩展的功能类似，不过请求扩展是基于装饰器实现的，其两者的执行顺序也是不同的。


安装： `pip install blinker`

# 信号的简单使用

In [1]:
from flask import Flask, signals

app = Flask(__name__)

# 信号函数
def single1(*args, **kwargs):
    print("信号1", args, kwargs)

# 给信号注册函数，在信号条件达到式触发，内置信号自动触发
signals.request_started.connect(single1)

@app.route('/')
def index():
    print('index')
    return "index"


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2022 10:47:18] "GET / HTTP/1.1" 200 -


信号1 (<Flask '__main__'>,) {}
index


# 内置信号

内置信号，会在请求进入或者返回的某一个点自动触发执行。
```python
request_started = _signals.signal('request-started')                # 请求到来前执行
request_finished = _signals.signal('request-finished')              # 请求结束后执行
 
before_render_template = _signals.signal('before-render-template')  # 模板渲染前执行
template_rendered = _signals.signal('template-rendered')            # 模板渲染后执行
 
got_request_exception = _signals.signal('got-request-exception')    # 请求执行出现异常时执行
 
request_tearing_down = _signals.signal('request-tearing-down')      # 请求执行完毕后自动执行（无论成功与否）
appcontext_tearing_down = _signals.signal('appcontext-tearing-down')# 应用上下文执行完毕后自动执行（无论成功与否）
 
appcontext_pushed = _signals.signal('appcontext-pushed')            # 应用上下文push时执行
appcontext_popped = _signals.signal('appcontext-popped')            # 应用上下文pop时执行
message_flashed = _signals.signal('message-flashed')                # 调用flask在其中添加数据时，自动触发
```

# 信号执行的三要素

- 信号存在，创建信号 _signals.signal
- 信号注册执行函数，connect进行注册
- 信号触发，请求过程中自动触发(内置信号)或者手动触发(自定义信号的send方法)

# 自定义信号

**普通信号**

```python
from flask.signals import _signals
# 自定义信号
xxxxx = _signals.signal('xxxxx')

def func(sender, *args, **kwargs):
    print(sender)

# 自定义信号中注册函数
xxxxx.connect(func)

# 触发信号
xxxxx.send('123123', k1='v1')
```

代码演示

In [2]:
from flask import Flask, signals
from flask.signals import _signals

app = Flask(__name__)

m_sgl = _signals.signal("m_sgl")


def m_sgl_test1(sender, **kwargs):
    print(sender, kwargs)


m_sgl.connect(m_sgl_test1)

@app.route('/')
def index():
    m_sgl.send([1, 2, 3], data={"a": 1})
    print('index')
    return "index"


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2022 11:08:20] "GET / HTTP/1.1" 200 -


信号1 (<Flask '__main__'>,) {}
[1, 2, 3] {'data': {'a': 1}}
index


**使用命名空间的信号**

```python
from blinker import Namespace  
# Namespace的作用：为了防止多人开发的时候，信号名字冲突的问题
zlspace = Namespace()
fire_signal = zlspace.signal('fire')
```

# 内置信号和请求扩展的触发顺序

- befor_first_request 请求扩展
- request_started 信号
- before_request 请求扩展
- before_render_template 信号
- template_rendered 信号
- after_request 请求扩展
- request_finished 信号
- 如果上述过程出错，触发错误处理信号  got_request_exception 

# 源码阅读

请求进来执行的app的`__call__`方法，从请求的入口进行分析，在`__call__`中调用的是wsgi_app方法

入口
```python
class Flask(_PackageBoundObject):
    def wsgi_app(self, environ: dict, start_response: t.Callable) -> t.Any:
        # ############### 上下文 ###############
        ctx = self.request_context(environ)
        error: t.Optional[BaseException] = None
        try:
            try:
                ctx.push()
                # ############### 请求处理过程 ###############
                response = self.full_dispatch_request()
            except Exception as e:
                error = e
                # ############### 请求处理过程中出现异常的处理 ###############
                response = self.handle_exception(e)
            except:  # noqa: B001
                error = sys.exc_info()[1]
                raise
            # ############### 返回response ###############
            return response(environ, start_response)
        finally:
            if self.should_ignore_error(error):
                error = None
            ctx.auto_pop(error)
```

request_started
```python
class Flask(_PackageBoundObject):

    def full_dispatch_request(self):
        # ###############1. befor_first_request 请求扩展 ###############
        self.try_trigger_before_first_request_functions()
        try:
            # ###############2. 触发request_started 信号 ###############
            request_started.send(self)
            # ###############3. before_request 请求扩展 ###############
            rv = self.preprocess_request()
            if rv is None:
                rv = self.dispatch_request()
        except Exception as e:
            rv = self.handle_user_exception(e)
        response = self.make_response(rv)

        # ###############6. request_finished 信号 ###############
        return self.finalize_request(rv)
    
     def full_dispatch_request(self) -> Response:
        self.try_trigger_before_first_request_functions()
        try:
            request_started.send(self)
            rv = self.preprocess_request()
            if rv is None:
                rv = self.dispatch_request()
        except Exception as e:
            rv = self.handle_user_exception(e)
        
    
    
    def try_trigger_before_first_request_functions(self) -> None:
        ###1.1 用一个flag来判断是否是第一条进来的请求，如果不是直接return
        if self._got_first_request:
            return
        with self._before_request_lock:
            if self._got_first_request:
                return
            for func in self.before_first_request_funcs:
                ###1.2 执行befor_first_request 请求扩展的所有函数
                self.ensure_sync(func)()
            ###1.3 当第一条请求执行完，将flag设置为True，后续的请求直接return
            self._got_first_request = True
            
    
    def preprocess_request(self) -> t.Optional[ResponseReturnValue]:
        names = (None, *reversed(request.blueprints))

        for name in names:
            if name in self.url_value_preprocessors:
                for url_func in self.url_value_preprocessors[name]:
                    url_func(request.endpoint, request.view_args)

        for name in names:
            if name in self.before_request_funcs:
                for before_func in self.before_request_funcs[name]:
                    ### 3.1 执行before_request请求扩展所有函数
                    rv = self.ensure_sync(before_func)()

                    if rv is not None:
                        return rv

        return None
    
     def finalize_request(
        self,
        rv: t.Union[ResponseReturnValue, HTTPException],
        from_error_handler: bool = False,
    ) -> Response:
        response = self.make_response(rv)
        try:
            # ###############6.1 ###############
            response = self.process_response(response)
            # ###############6.2 request_finished 信号 ###############
            request_finished.send(self, response=response)
        except Exception:
            if not from_error_handler:
                raise
            self.logger.exception(
                "Request finalizing failed with an error while handling an error"
            )
        return response
    
    
    def process_response(self, response: Response) -> Response:
        ctx = _request_ctx_stack.top

        for func in ctx._after_request_functions:
            response = self.ensure_sync(func)(response)

        for name in chain(request.blueprints, (None,)):
            if name in self.after_request_funcs:
                # ###############6.1.1 after_request请求扩展###############
                for func in reversed(self.after_request_funcs[name]):
                    response = self.ensure_sync(func)(response)

        if not self.session_interface.is_null_session(ctx.session):
            # ###############6.1.2 save_session() ###############
            self.session_interface.save_session(self, ctx.session, response)

        return response
```

```python
def render_template(
    template_name_or_list: t.Union[str, t.List[str]], **context: t.Any
) -> str:
    ctx = _app_ctx_stack.top
    ctx.app.update_template_context(context)
    return _render(
        ctx.app.jinja_env.get_or_select_template(template_name_or_list),
        context,
        ctx.app,
    )

def _render(template: Template, context: dict, app: "Flask") -> str:
    # ###############4. before_render_template 信号 ###############
    before_render_template.send(app, template=template, context=context)
    rv = template.render(context)
    # ###############5. template_rendered 信号 ###############
    template_rendered.send(app, template=template, context=context)
    return rv
```